In [28]:
# https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

In [10]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [19]:
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5
print(lstm, inputs)

# initialize the hidden state.
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))
print(hidden)

LSTM(3, 3) [tensor([[1.1393, 0.0582, 0.8405]]), tensor([[-0.3500,  0.3181, -0.1518]]), tensor([[ 0.4300, -1.1828, -0.7735]]), tensor([[-0.9992, -0.5900, -1.7509]]), tensor([[-1.4686, -0.8066,  0.1316]])]
(tensor([[[-0.3633,  0.4898, -0.2826]]]), tensor([[[-1.4921,  2.7120, -2.0804]]]))


In [11]:
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)

# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

tensor([[[-0.0187,  0.1713, -0.2944]],

        [[-0.3521,  0.1026, -0.2971]],

        [[-0.3191,  0.0781, -0.1957]],

        [[-0.1634,  0.0941, -0.1637]],

        [[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward>)
(tensor([[[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward>), tensor([[[-0.9825,  0.4715, -0.0633]]], grad_fn=<StackBackward>))


In [12]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [13]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [17]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[1][0], word_to_ix)
    print(inputs)
    tag_scores = model(inputs)
    print(tag_scores)

tensor([5, 6, 7, 8])
tensor([[-1.3328, -1.1364, -0.8788],
        [-1.3647, -1.0470, -0.9325],
        [-1.4329, -0.9545, -0.9771],
        [-1.5074, -0.9057, -0.9828]])


In [15]:
training_data

[(['The', 'dog', 'ate', 'the', 'apple'], ['DET', 'NN', 'V', 'DET', 'NN']),
 (['Everybody', 'read', 'that', 'book'], ['NN', 'V', 'DET', 'NN'])]

In [20]:
for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        print(sentence_in)
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0,

tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0, 1, 2, 3, 4])
tensor([5, 6, 7, 8])
tensor([0,

In [ ]:
# https://discuss.pytorch.org/t/example-of-many-to-one-lstm/1728/2

In [21]:
import torch
import torch.nn as nn
from torch.autograd import Variable

time_steps = 10
batch_size = 3
in_size = 5
classes_no = 7

model = nn.LSTM(in_size, classes_no, 2)
input_seq = Variable(torch.randn(time_steps, batch_size, in_size))
output_seq, _ = model(input_seq)
last_output = output_seq[-1]

loss = nn.CrossEntropyLoss()
target = Variable(torch.LongTensor(batch_size).random_(0, classes_no-1))
err = loss(last_output, target)
err.backward()

In [27]:
input_seq.shape, input_seq, target

(torch.Size([10, 3, 5]),
 tensor([[[-0.1973,  0.3036, -1.9171, -0.1702,  0.6982],
          [ 0.2959, -1.7451, -1.1650, -1.2542,  0.2417],
          [-0.5246, -0.5836, -0.9227, -0.3253,  1.0962]],
 
         [[ 0.3849, -1.4344, -0.7832, -0.7015,  0.2843],
          [ 0.1317,  0.4890, -1.2848, -0.3952, -0.3694],
          [-0.3983,  0.4598,  0.0107, -0.0460,  0.5487]],
 
         [[ 0.0473, -1.1824, -1.0940, -0.6191,  0.0983],
          [-1.9932,  1.1927,  0.7687,  1.5591,  1.5513],
          [-0.8843,  0.5306,  0.3547, -0.7996, -0.1471]],
 
         [[-2.3783, -0.4609, -0.3070, -2.0573,  0.7002],
          [ 0.5472,  1.1474, -0.6696, -0.7714, -0.2665],
          [ 0.0449,  0.4412, -0.0707,  0.4662,  0.0490]],
 
         [[-0.3228, -0.5578,  0.8521,  2.3545, -0.6657],
          [ 0.4297, -1.0645,  1.3067, -0.8167, -1.0331],
          [-0.3161, -0.6191, -1.2703, -1.1421,  0.0410]],
 
         [[ 1.0469,  0.1030,  1.8462,  0.4964,  0.4062],
          [ 1.0015, -0.9535, -0.1564,  0.2379, -